In [1]:
import datetime
import glob
import gzip
import json
        
root_dir = "/Users/nshelton/Library/Mobile Documents/com~apple~CloudDocs/location-history.json"


f = open(root_dir, 'rb')
fileContent = f.read()
rawFileContent = fileContent.decode('utf-8')
data = json.loads(rawFileContent)
f.close()

print(len(data))



39475


In [2]:
import re

samples = []

# Regular expression pattern to extract latitude and longitude
pattern = re.compile(r'geo:([-+]?\d*\.\d+|\d+),([-+]?\d*\.\d+|\d+)')

for location in data:
    
    place_location = ''

    if 'visit' in location:
      place_location = location['visit']['topCandidate']['placeLocation']
      
    if 'activity' in location:
      place_location = location['activity']['start']
     
    if 'timelinePath' in location:
      place_location = location['timelinePath'][0]['point']

    timestamp = location['startTime']


    if place_location == '':
        # print('No location data')
        continue

    match = pattern.match(place_location)
    if match:
        lat = float(match.group(1))
        lon = float(match.group(2))
        # print(lat, lon, timestamp)
        samples.append((lat, lon, timestamp))



In [3]:
len(samples)


39405

In [4]:
import sqlite3

def initialize_database(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS locations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            time TEXT,
            lat Decimal(8,6),
            lng Decimal(9,6)
        )
    ''')
    conn.commit()
    return conn

db_path = 'locations.db'
conn = initialize_database(db_path)

cursor = conn.cursor()
for sample in samples:

    cursor.execute('''
      INSERT INTO locations (time, lat, lng)
      VALUES (?, ?, ?)
      ''', (sample[2], sample[0], sample[1]))
conn.commit() 


